In [1]:
import numpy as np
import cv2
import glob
import pickle
# import argparse

In [2]:
NUM_IMAGES = 1000

In [3]:
class StereoCalibration(object):
    def __init__(self, filepath, show_corner=False):
        # termination criteria
        self.criteria = (cv2.TERM_CRITERIA_EPS +
                         cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        self.criteria_cal = (cv2.TERM_CRITERIA_EPS +
                             cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-5)
        self.M1 = None
        self.M2 = None
        self.show_corner = show_corner

        # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
        self.objp = np.zeros((9*6, 3), np.float32)
        self.objp[:, :2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2)

        # Arrays to store object points and image points from all the images.
        self.objpoints = []  # 3d point in real world space
        self.imgpoints_l = []  # 2d points in image plane.
        self.imgpoints_r = []  # 2d points in image plane.

        self.cal_path = filepath
        self.read_images(self.cal_path)

    def read_images(self, cal_path):
        images_right = glob.glob(cal_path + 'right/' + '*r.png')
        images_left = glob.glob(cal_path + 'left/' + '*l.png')
        print(f"Found {len(images_left)} left camera images")
        print(f"Found {len(images_right)} right camera images")
        images_left.sort()
        images_right.sort()

        for i, fname in enumerate(images_right):
            print("processing pair {}...".format(i))
            img_l = cv2.imread(images_left[i])
            img_r = cv2.imread(images_right[i])

            gray_l = cv2.cvtColor(img_l, cv2.COLOR_BGR2GRAY)
            gray_r = cv2.cvtColor(img_r, cv2.COLOR_BGR2GRAY)

            # Find the chess board corners
            ret_l, corners_l = cv2.findChessboardCorners(gray_l, (9, 6), None)
            ret_r, corners_r = cv2.findChessboardCorners(gray_r, (9, 6), None)

            if ret_l and ret_r:
                # If found, add object points, image points (after refining them)
                self.objpoints.append(self.objp)
            
                rt = cv2.cornerSubPix(gray_l, corners_l, (11, 11),
                                      (-1, -1), self.criteria)
                self.imgpoints_l.append(corners_l)

                # Draw and display the corners
                ret_l = cv2.drawChessboardCorners(img_l, (9, 6),
                                                  corners_l, ret_l)
                if self.show_corner:
                    cv2.imshow(images_left[i], img_l)
                    cv2.waitKey(500)

                rt = cv2.cornerSubPix(gray_r, corners_r, (11, 11),
                                      (-1, -1), self.criteria)
                self.imgpoints_r.append(corners_r)

                # Draw and display the corners
                ret_r = cv2.drawChessboardCorners(img_r, (9, 6),
                                                  corners_r, ret_r)
                if self.show_corner:
                    cv2.imshow(images_right[i], img_r)
                    cv2.waitKey(500)
            img_shape = gray_l.shape[::-1]

        print(len(self.objpoints))
        print(len(self.imgpoints_l))
        print(len(self.imgpoints_r))
        
        rt, self.M1, self.d1, self.r1, self.t1 = cv2.calibrateCamera(
            self.objpoints[:NUM_IMAGES], self.imgpoints_l[:NUM_IMAGES], img_shape, None, None)
        rt, self.M2, self.d2, self.r2, self.t2 = cv2.calibrateCamera(
            self.objpoints[:NUM_IMAGES], self.imgpoints_r[:NUM_IMAGES], img_shape, None, None)

        print("finish calibrating")
        self.camera_model = self.stereo_calibrate(img_shape)

    def stereo_calibrate(self, dims):
        flags = 0
        flags |= cv2.CALIB_FIX_INTRINSIC
        # flags |= cv2.CALIB_FIX_PRINCIPAL_POINT
        flags |= cv2.CALIB_USE_INTRINSIC_GUESS
        # flags |= cv2.CALIB_USE_EXTRINSIC_GUESS
        flags |= cv2.CALIB_FIX_FOCAL_LENGTH
        # flags |= cv2.CALIB_FIX_ASPECT_RATIO
        flags |= cv2.CALIB_ZERO_TANGENT_DIST
        # flags |= cv2.CALIB_RATIONAL_MODEL
        # flags |= cv2.CALIB_SAME_FOCAL_LENGTH
        # flags |= cv2.CALIB_FIX_K3
        # flags |= cv2.CALIB_FIX_K4
        # flags |= cv2.CALIB_FIX_K5

        stereocalib_criteria = (cv2.TERM_CRITERIA_MAX_ITER +
                                cv2.TERM_CRITERIA_EPS, 100, 1e-5)
        
        print("start modelling")
        ret, M1, d1, M2, d2, R, T, E, F = cv2.stereoCalibrate(
            self.objpoints[:NUM_IMAGES], self.imgpoints_l[:NUM_IMAGES],
            self.imgpoints_r[:NUM_IMAGES], self.M1, self.d1, self.M2,
            self.d2, dims,
            criteria=stereocalib_criteria, flags=flags)

        print('Intrinsic_mtx_1', M1)
        print('dist_1', d1)
        print('Intrinsic_mtx_2', M2)
        print('dist_2', d2)
        print('R', R)
        print('T', T)
        print('E', E)
        print('F', F)

        # for i in range(len(self.r1)):
        #     print("--- pose[", i+1, "] ---")
        #     self.ext1, _ = cv2.Rodrigues(self.r1[i])
        #     self.ext2, _ = cv2.Rodrigues(self.r2[i])
        #     print('Ext1', self.ext1)
        #     print('Ext2', self.ext2)

        print('')

        camera_model = dict([('M1', M1), ('M2', M2), ('dist1', d1),
                            ('dist2', d2), ('rvecs1', self.r1),
                            ('rvecs2', self.r2), ('R', R), ('T', T),
                            ('E', E), ('F', F)])

        cv2.destroyAllWindows()
        print("finish modeling")
        
#         with open("camera_model.json", "w") as outfile:
#             json.dump(dictionary, outfile, indent=4)
        
        return camera_model

In [ ]:
if __name__ == '__main__':
#     parser = argparse.ArgumentParser()
#     parser.add_argument('filepath', help='String Filepath')
#     args = parser.parse_args()
    cal_data = StereoCalibration("./stereo/calib/")
    with open("camera_model.pickle", "wb") as outfile:
        pickle.dump(cal_data, outfile)
    StereoCalibration("./stereo/calib/", False)

Found 12968 left camera images
Found 12968 right camera images
processing pair 0...
processing pair 1...
processing pair 2...
processing pair 3...
processing pair 4...
processing pair 5...
processing pair 6...
processing pair 7...
processing pair 8...
processing pair 9...
processing pair 10...
processing pair 11...
processing pair 12...
processing pair 13...
processing pair 14...
processing pair 15...
processing pair 16...
processing pair 17...
processing pair 18...
processing pair 19...
processing pair 20...
processing pair 21...
processing pair 22...
processing pair 23...
processing pair 24...
processing pair 25...
processing pair 26...
processing pair 27...
processing pair 28...
processing pair 29...
processing pair 30...
processing pair 31...
processing pair 32...
processing pair 33...
processing pair 34...
processing pair 35...
processing pair 36...
processing pair 37...
processing pair 38...
processing pair 39...
processing pair 40...
processing pair 41...
processing pair 42...
p

processing pair 360...
processing pair 361...
processing pair 362...
processing pair 363...
processing pair 364...
processing pair 365...
processing pair 366...
processing pair 367...
processing pair 368...
processing pair 369...
processing pair 370...
processing pair 371...
processing pair 372...
processing pair 373...
processing pair 374...
processing pair 375...
processing pair 376...
processing pair 377...
processing pair 378...
processing pair 379...
processing pair 380...
processing pair 381...
processing pair 382...
processing pair 383...
processing pair 384...
processing pair 385...
processing pair 386...
processing pair 387...
processing pair 388...
processing pair 389...
processing pair 390...
processing pair 391...
processing pair 392...
processing pair 393...
processing pair 394...
processing pair 395...
processing pair 396...
processing pair 397...
processing pair 398...
processing pair 399...
processing pair 400...
processing pair 401...
processing pair 402...
processing 

processing pair 719...
processing pair 720...
processing pair 721...
processing pair 722...
processing pair 723...
processing pair 724...
processing pair 725...
processing pair 726...
processing pair 727...
processing pair 728...
processing pair 729...
processing pair 730...
processing pair 731...
processing pair 732...
processing pair 733...
processing pair 734...
processing pair 735...
processing pair 736...
processing pair 737...
processing pair 738...
processing pair 739...
processing pair 740...
processing pair 741...
processing pair 742...
processing pair 743...
processing pair 744...
processing pair 745...
processing pair 746...
processing pair 747...
processing pair 748...
processing pair 749...
processing pair 750...
processing pair 751...
processing pair 752...
processing pair 753...
processing pair 754...
processing pair 755...
processing pair 756...
processing pair 757...
processing pair 758...
processing pair 759...
processing pair 760...
processing pair 761...
processing 

processing pair 1076...
processing pair 1077...
processing pair 1078...
processing pair 1079...
processing pair 1080...
processing pair 1081...
processing pair 1082...
processing pair 1083...
processing pair 1084...
processing pair 1085...
processing pair 1086...
processing pair 1087...
processing pair 1088...
processing pair 1089...
processing pair 1090...
processing pair 1091...
processing pair 1092...
processing pair 1093...
processing pair 1094...
processing pair 1095...
processing pair 1096...
processing pair 1097...
processing pair 1098...
processing pair 1099...
processing pair 1100...
processing pair 1101...
processing pair 1102...
processing pair 1103...
processing pair 1104...
processing pair 1105...
processing pair 1106...
processing pair 1107...
processing pair 1108...
processing pair 1109...
processing pair 1110...
processing pair 1111...
processing pair 1112...
processing pair 1113...
processing pair 1114...
processing pair 1115...
processing pair 1116...
processing pair 

processing pair 1421...
processing pair 1422...
processing pair 1423...
processing pair 1424...
processing pair 1425...
processing pair 1426...
processing pair 1427...
processing pair 1428...
processing pair 1429...
processing pair 1430...
processing pair 1431...
processing pair 1432...
processing pair 1433...
processing pair 1434...
processing pair 1435...
processing pair 1436...
processing pair 1437...
processing pair 1438...
processing pair 1439...
processing pair 1440...
processing pair 1441...
processing pair 1442...
processing pair 1443...
processing pair 1444...
processing pair 1445...
processing pair 1446...
processing pair 1447...
processing pair 1448...
processing pair 1449...
processing pair 1450...
processing pair 1451...
processing pair 1452...
processing pair 1453...
processing pair 1454...
processing pair 1455...
processing pair 1456...
processing pair 1457...
processing pair 1458...
processing pair 1459...
processing pair 1460...
processing pair 1461...
processing pair 

...

processing pair 11380...
processing pair 11381...
processing pair 11382...
processing pair 11383...
processing pair 11384...
processing pair 11385...
processing pair 11386...
processing pair 11387...
processing pair 11388...
processing pair 11389...
processing pair 11390...
processing pair 11391...
processing pair 11392...
processing pair 11393...
processing pair 11394...
processing pair 11395...
processing pair 11396...
processing pair 11397...
processing pair 11398...
processing pair 11399...
processing pair 11400...
processing pair 11401...
processing pair 11402...
processing pair 11403...
processing pair 11404...
processing pair 11405...
processing pair 11406...
processing pair 11407...
processing pair 11408...
processing pair 11409...
processing pair 11410...
processing pair 11411...
processing pair 11412...
processing pair 11413...
processing pair 11414...
processing pair 11415...
processing pair 11416...
processing pair 11417...
processing pair 11418...
processing pair 11419...


processing pair 11711...
processing pair 11712...
processing pair 11713...
processing pair 11714...
processing pair 11715...
processing pair 11716...
processing pair 11717...
processing pair 11718...
processing pair 11719...
processing pair 11720...
processing pair 11721...
processing pair 11722...
processing pair 11723...
processing pair 11724...
processing pair 11725...
processing pair 11726...
processing pair 11727...
processing pair 11728...
processing pair 11729...
processing pair 11730...
processing pair 11731...
processing pair 11732...
processing pair 11733...
processing pair 11734...
processing pair 11735...
processing pair 11736...
processing pair 11737...
processing pair 11738...
processing pair 11739...
processing pair 11740...
processing pair 11741...
processing pair 11742...
processing pair 11743...
processing pair 11744...
processing pair 11745...
processing pair 11746...
processing pair 11747...
processing pair 11748...
processing pair 11749...
processing pair 11750...


processing pair 12039...
processing pair 12040...
processing pair 12041...
processing pair 12042...
processing pair 12043...
processing pair 12044...
processing pair 12045...
processing pair 12046...
processing pair 12047...
processing pair 12048...
processing pair 12049...
processing pair 12050...
processing pair 12051...
processing pair 12052...
processing pair 12053...
processing pair 12054...
processing pair 12055...
processing pair 12056...
processing pair 12057...
processing pair 12058...
processing pair 12059...
processing pair 12060...
processing pair 12061...
processing pair 12062...
processing pair 12063...
processing pair 12064...
processing pair 12065...
processing pair 12066...
processing pair 12067...
processing pair 12068...
processing pair 12069...
processing pair 12070...
processing pair 12071...
processing pair 12072...
processing pair 12073...
processing pair 12074...
processing pair 12075...
processing pair 12076...
processing pair 12077...
processing pair 12078...


processing pair 12374...
processing pair 12375...
processing pair 12376...
processing pair 12377...
processing pair 12378...
processing pair 12379...
processing pair 12380...
processing pair 12381...
processing pair 12382...
processing pair 12383...
processing pair 12384...
processing pair 12385...
processing pair 12386...
processing pair 12387...
processing pair 12388...
processing pair 12389...
processing pair 12390...
processing pair 12391...
processing pair 12392...
processing pair 12393...
processing pair 12394...
processing pair 12395...
processing pair 12396...
processing pair 12397...
processing pair 12398...
processing pair 12399...
processing pair 12400...
processing pair 12401...
processing pair 12402...
processing pair 12403...
processing pair 12404...
processing pair 12405...
processing pair 12406...
processing pair 12407...
processing pair 12408...
processing pair 12409...
processing pair 12410...
processing pair 12411...
processing pair 12412...
processing pair 12413...


processing pair 12705...
processing pair 12706...
processing pair 12707...
processing pair 12708...
processing pair 12709...
processing pair 12710...
processing pair 12711...
processing pair 12712...
processing pair 12713...
processing pair 12714...
processing pair 12715...
processing pair 12716...
processing pair 12717...
processing pair 12718...
processing pair 12719...
processing pair 12720...
processing pair 12721...
processing pair 12722...
processing pair 12723...
processing pair 12724...
processing pair 12725...
processing pair 12726...
processing pair 12727...
processing pair 12728...
processing pair 12729...
processing pair 12730...
processing pair 12731...
processing pair 12732...
processing pair 12733...
processing pair 12734...
processing pair 12735...
processing pair 12736...
processing pair 12737...
processing pair 12738...
processing pair 12739...
processing pair 12740...
processing pair 12741...
processing pair 12742...
processing pair 12743...
processing pair 12744...
